In [1]:
import numpy as np
import seaborn as sn
from imblearn.over_sampling import SMOTE 
import keras
from keras.utils import np_utils
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential
from keras import losses
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import class_weight
from sklearn.metrics import recall_score, confusion_matrix, multilabel_confusion_matrix, precision_score, precision_recall_curve, average_precision_score, make_scorer
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from pandas import DataFrame
import xlsxwriter
import time

IBIS = pd.ExcelFile("SA_and_CT_AALandfROI_08272019.xlsx")

#Get the sheets for scaling
IBIS_CT = IBIS.sheet_names[9]
CT = IBIS.parse(IBIS_CT)

#Get rid of subject names to only have features now. #Need to remove ROIs. They don't convert to floats.
#Get rid of ctx_rh_Medial_wall and ctx_lh_Medial_wall, not needed for analysis.
#Scaling
IBIS_CT_features = CT.loc[:, CT.columns] 
IBIS_CT_features = IBIS_CT_features.drop(['Case','Visit','ROI42','ROI117'], axis=1)  
scalerCT = MinMaxScaler()
scaled_CT_data = scalerCT.fit_transform(IBIS_CT_features)

#Get the sheets for scaling
IBIS_SA = IBIS.sheet_names[10]
SA = IBIS.parse(IBIS_SA)

#Get rid of subject names to only have features now. #Need to remove ROIs. They don't convert to floats.
#Get rid of ctx_rh_Medial_wall and ctx_lh_Medial_wall, not needed for analysis.
#Scaling

IBIS_SA_features = SA.loc[:, SA.columns] 
IBIS_SA_features = IBIS_SA_features.drop(['Case','Visit','ROI42','ROI117'], axis=1)  
scalerSA = MinMaxScaler()
scaled_SA_data = scalerSA.fit_transform(IBIS_SA_features)

file = pd.ExcelFile("All CTSA 2-1yr.xlsx") #Only includes IBIS subjects that have 1,2 year CT,SA.

ct1y_sheet = file.sheet_names[0]
ct1y_data = file.parse(ct1y_sheet)
ct1y_data_features = ct1y_data.loc[:, ct1y_data.columns] 
ct1y_data_features = ct1y_data_features.drop(['ROI', 11142, 12142], axis=1)  #Already removed medial walls.

ct2y_sheet = file.sheet_names[1]
ct2y_data = file.parse(ct2y_sheet)
ct2y_data_features = ct2y_data.loc[:, ct2y_data.columns] 
ct2y_data_features = ct2y_data_features.drop(['ROI', 11142, 12142], axis=1)  #Already removed medial walls.

sa1y_sheet = file.sheet_names[2]
sa1y_data = file.parse(sa1y_sheet)
sa1y_data_features = sa1y_data.loc[:, sa1y_data.columns] 
sa1y_data_features = sa1y_data_features.drop(['ROI', 11142, 12142], axis=1)  #Already removed medial walls.

sa2y_sheet = file.sheet_names[3]
sa2y_data = file.parse(sa2y_sheet)
sa2y_data_features = sa2y_data.loc[:, sa2y_data.columns] 
sa2y_data_features = sa2y_data_features.drop(['ROI', 11142, 12142], axis=1)  #Already removed medial walls.

scaled_ct1y = scalerSA.transform(ct1y_data_features)
scaled_ct2y = scalerSA.transform(ct2y_data_features)

print(ct1y_data_features.shape)
print(ct2y_data_features.shape)
print(sa1y_data_features.shape)
print(sa2y_data_features.shape)

/nas/longleaf/apps/anaconda/2019.10.ood/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


(585, 148)
(585, 148)
(585, 148)
(585, 148)


In [2]:
def create_model(dropout=0.1, layer1_size=110, layer2_size=20, encoded_layer_size=15):
    # create model
    model = Sequential()
    model.add(Dense(layer1_size, input_dim=148, activation='tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(layer2_size, activation='tanh'))
    model.add(Dense(encoded_layer_size, activation='tanh'))
    model.add(Dense(layer2_size, activation='tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(layer1_size, activation='tanh'))
    model.add(Dense(148,activation='tanh'))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer='adam')
    return model

In [3]:
# create model
model = KerasRegressor(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 15, 20, 25, 30]
epochs = [50, 100, 150] 
dropout_rate = [0.15, 0.2, 0.25]
#Hidden layer sizes
hidden_size1 = [90, 100, 110] 
hidden_size2 = [25, 30, 35, 40] 
encoded_layer_size_array = [15,20,25]

param_grid = dict(batch_size=batch_size, epochs=epochs, dropout=dropout_rate, layer1_size=hidden_size1, layer2_size=hidden_size2, encoded_layer_size=encoded_layer_size_array)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_ct1y,scaled_ct2y)

/nas/longleaf/apps/anaconda/2019.10.ood/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [4]:
df = pd.DataFrame(grid_result.cv_results_)
df.to_excel("negMeanAbsoluteErrorCT.xlsx")
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [5]:
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_dropout,param_encoded_layer_size,param_epochs,param_layer1_size,param_layer2_size,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.053921,0.269749,0.081891,0.008718,10,0.15,15,50,90,25,"{'batch_size': 10, 'dropout': 0.15, 'encoded_l...",-0.002396,-0.002025,-0.002597,-0.002978,-0.003527,-0.002705,0.000514,1059
1,3.193261,0.416799,0.083444,0.045273,10,0.15,15,50,90,30,"{'batch_size': 10, 'dropout': 0.15, 'encoded_l...",-0.002659,-0.002357,-0.002523,-0.003314,-0.003292,-0.002829,0.000399,1166
2,3.494461,0.642031,0.126624,0.073174,10,0.15,15,50,90,35,"{'batch_size': 10, 'dropout': 0.15, 'encoded_l...",-0.002287,-0.002388,-0.002566,-0.003229,-0.003122,-0.002718,0.000385,1080
3,3.516383,0.587171,0.103072,0.067731,10,0.15,15,50,90,40,"{'batch_size': 10, 'dropout': 0.15, 'encoded_l...",-0.002276,-0.002365,-0.002701,-0.003509,-0.003321,-0.002834,0.000498,1170
4,3.366699,0.593824,0.099258,0.055989,10,0.15,15,50,100,25,"{'batch_size': 10, 'dropout': 0.15, 'encoded_l...",-0.001784,-0.001876,-0.002517,-0.003169,-0.002975,-0.002464,0.000560,384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,3.990674,0.686145,0.100208,0.042651,30,0.25,25,150,100,40,"{'batch_size': 30, 'dropout': 0.25, 'encoded_l...",-0.001746,-0.001848,-0.002412,-0.002986,-0.002866,-0.002372,0.000508,73
1616,3.859505,0.614858,0.088859,0.039170,30,0.25,25,150,110,25,"{'batch_size': 30, 'dropout': 0.25, 'encoded_l...",-0.001815,-0.001889,-0.002347,-0.002931,-0.002979,-0.002392,0.000495,123
1617,4.100459,0.721476,0.084658,0.026327,30,0.25,25,150,110,30,"{'batch_size': 30, 'dropout': 0.25, 'encoded_l...",-0.002009,-0.001803,-0.002412,-0.002950,-0.002876,-0.002410,0.000456,184
1618,4.163100,1.028410,0.084866,0.023138,30,0.25,25,150,110,35,"{'batch_size': 30, 'dropout': 0.25, 'encoded_l...",-0.001849,-0.001946,-0.002287,-0.003007,-0.002933,-0.002404,0.000485,166


In [6]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.002279 using {'batch_size': 30, 'dropout': 0.15, 'encoded_layer_size': 25, 'epochs': 150, 'layer1_size': 90, 'layer2_size': 25}
-0.002705 (0.000514) with: {'batch_size': 10, 'dropout': 0.15, 'encoded_layer_size': 15, 'epochs': 50, 'layer1_size': 90, 'layer2_size': 25}
-0.002829 (0.000399) with: {'batch_size': 10, 'dropout': 0.15, 'encoded_layer_size': 15, 'epochs': 50, 'layer1_size': 90, 'layer2_size': 30}
-0.002718 (0.000385) with: {'batch_size': 10, 'dropout': 0.15, 'encoded_layer_size': 15, 'epochs': 50, 'layer1_size': 90, 'layer2_size': 35}
-0.002834 (0.000498) with: {'batch_size': 10, 'dropout': 0.15, 'encoded_layer_size': 15, 'epochs': 50, 'layer1_size': 90, 'layer2_size': 40}
-0.002464 (0.000560) with: {'batch_size': 10, 'dropout': 0.15, 'encoded_layer_size': 15, 'epochs': 50, 'layer1_size': 100, 'layer2_size': 25}
-0.002648 (0.000515) with: {'batch_size': 10, 'dropout': 0.15, 'encoded_layer_size': 15, 'epochs': 50, 'layer1_size': 100, 'layer2_size': 30}
-0.002846 (0.00